The use case of this python script is to get the hubspot campaign using API key manually. 

key problem - currently in stitch API table stitch could not provide us the data of campaign level. eventho there is campaign table but it is actually email campaign level table. 

Import all the libaries and app token ( hubspot)

In [ ]:
# Import libraries
import requests
import pandas as pd
import pandas_gbq as bq
import pydata_google_auth

# Set private app token

In [ ]:
import json
import pandas as pd
import requests

url = "https://api.hubapi.com/marketing-emails/v1/emails/"
all_objects = []
offset = 0
limit = 10  # This seems to be the default limit from the API

headers = {
    'accept': "application/json",
    'authorization': f'Bearer {key}'
}

try:
    # Loop until we get all records
    while True:
        # Add offset to querystring
        querystring = {
            "properties": ["campaign"],
            "offset": offset
        }
        
        response = requests.request("GET", url, headers=headers, params=querystring)
        response.raise_for_status()
        data = response.json()
        
        objects = data['objects']
        all_objects.extend(objects)
        
        print(f"Fetched {len(objects)} objects. Total so far: {len(all_objects)}")
        
        # Check if we've got all records
        if len(all_objects) >= data['total']:
            break
            
        # Update offset for next batch
        offset += limit

    print(f"\nTotal records fetched: {len(all_objects)}")

    # Create list of dictionaries for DataFrame
    campaign_pairs = []
    for obj in all_objects:
        campaign = obj.get('campaign', '')
        campaign_ids = obj.get('allEmailCampaignIds', [])
        
        if campaign_ids:
            for email_id in campaign_ids:
                campaign_pairs.append({
                    'campaign_id': campaign,
                    'email_campaign_id': email_id,
                    'campaign_name': obj.get('campaignName', '')
                })

    # Convert to DataFrame and save
    df = pd.DataFrame(campaign_pairs)
    print(f"Created DataFrame with {len(df)} rows")
    
    df.to_csv("emailid_w_campaignid.csv", index=False)
    print("Data successfully exported to emailid_w_campaignid.csv")

except requests.exceptions.RequestException as e:
    print(f"Error making API request: {e}")
except json.JSONDecodeError as e:
    print(f"Error parsing JSON response: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")